In [ ]:
pip install protobuf>=3.20.3

In [ ]:
%pip install git+https://github.com/espnet/espnet
%pip install -q espnet_model_zoo

In [ ]:
# Downloading one utterance from real noisy speech of CHiME4
!gdown --id 1SmrN5NFSg6JuQSs2sfy3ehD8OIcqK6wS -O /content/M05_440C0213_PED_REAL.wav
import os

import soundfile
from IPython.display import display, Audio
mixwav_mc, sr = soundfile.read("/content/M05_440C0213_PED_REAL.wav")
# mixwav.shape: num_samples, num_channels
mixwav_sc = mixwav_mc[:,4]
display(Audio(mixwav_mc.T, rate=sr))

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1SmrN5NFSg6JuQSs2sfy3ehD8OIcqK6wS
To: /content/M05_440C0213_PED_REAL.wav
100% 932k/932k [00:00<00:00, 112MB/s]


In [ ]:

from espnet_model_zoo.downloader import ModelDownloader

d = ModelDownloader()
cfg = d.download_and_unpack("espnet/Wangyou_Zhang_chime4_enh_train_enh_conv_tasnet_raw")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:

import sys
import soundfile
from espnet2.bin.enh_inference import SeparateSpeech


separate_speech = {}

enh_model_sc = SeparateSpeech(
  train_config=cfg["train_config"],
  model_file=cfg["model_file"],
  # for segment-wise process on long speech
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=4,
  normalize_output_wav=True,
  device="cpu",
)

In [ ]:
# playing the enhanced single-channel speech
wave = enh_model_sc(mixwav_sc[None, ...], sr)
print("Input real noisy speech", flush=True)
display(Audio(mixwav_sc, rate=sr))
print("Enhanced speech", flush=True)
display(Audio(wave[0].squeeze(), rate=sr))

Input real noisy speech


Enhanced speech


In [ ]:
from google.colab import files
from IPython.display import display, Audio
import soundfile

uploaded = files.upload()

for file_name in uploaded.keys():
  speech, rate = soundfile.read(file_name)
  assert rate == sr, "mismatch in sampling rate"
  wave = enh_model_sc(speech[None, ...], sr)
  print(f"Your input speech {file_name}", flush=True)
  display(Audio(speech, rate=sr))
  print(f"Enhanced speech for {file_name}", flush=True)
  display(Audio(wave[0].squeeze(), rate=sr))



KeyboardInterrupt: ignored

In [ ]:
import numpy as np

def snr(clean_signal, noisy_signal):
    """
    Calculate the Signal-to-Noise Ratio (SNR) between two audio signals.
    """
    clean_signal = np.asarray(clean_signal)
    noisy_signal = np.asarray(noisy_signal)

    # Check if the audio signals have the same shape
    if clean_signal.shape != noisy_signal.shape:
        raise ValueError("Both signals must have the same shape.")

    # Calculate the power of the clean signal
    clean_signal_power = np.sum(np.square(clean_signal))

    # Calculate the power of the noise signal
    noise_signal_power = np.sum(np.square(clean_signal - noisy_signal))

    # Calculate the SNR in decibels
    snr_db = 10 * np.log10(clean_signal_power / noise_signal_power)

    return snr_db


In [ ]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa
clean_signal, sr = librosa.load("/content/Clean.wav", sr=None)
noisy_signal, sr = librosa.load("/content/M05_440C0213_PED_REAL.wav", sr=None)

# Calculate the SNR
snr_value = snr(clean_signal, noisy_signal)

# Print the SNR
print("SNR:", snr_value, "dB")


SNR: 4.700098633766174 dB


In [ ]:
import numpy as np

def snr(x, y):
    """Calculate the signal-to-noise ratio (SNR) of two audio signals."""
    x = np.asarray(x)
    y = np.asarray(y)
    if len(x) != len(y):
        raise ValueError("Inputs must have the same length.")
    # Calculate the power of the clean signal and the noise
    clean_power = np.sum(x ** 2)
    noise_power = np.sum((x - y) ** 2)
    # Calculate the SNR in decibels (dB)
    snr_db = 10 * np.log10(clean_power / noise_power)
    return snr_db

# Load the clean and noisy audio signals
clean_signal, sr = librosa.load("/content/Clean.wav", sr=None)
noisy_signal, sr = librosa.load("/content/M05_440C0213_PED_REAL.wav", sr=None)

# Calculate the SNR
snr_db = snr( clean_signal,noisy_signal)

# Print the SNR
print("SNR:", snr_db, "dB")


SNR: 4.700098633766174 dB


In [ ]:
#@title Speech Enhancement model { run: "auto" }

fs = 16000 #@param {type:"integer"}
tag = "espnet/Wangyou_Zhang_chime4_enh_train_enh_beamformer_mvdr_raw" #@param ["espnet/Wangyou_Zhang_chime4_enh_train_enh_beamformer_mvdr_raw", "espnet/Wangyou_Zhang_chime4_enh_train_enh_dc_crn_mapping_snr_raw", "lichenda/chime4_fasnet_dprnn_tac", "https://zenodo.org/record/6025881/files/enh_train_enh_beamformer_mvdr_raw_valid.si_snr.ave.zip"]

In [ ]:
#@title Model
import sys
import soundfile
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.enh_inference import SeparateSpeech

d = ModelDownloader()

cfg = d.download_and_unpack(tag)
enh_model_mc = SeparateSpeech(
  train_config=cfg["train_config"],
  model_file=cfg["model_file"],
  # for segment-wise process on long speech
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=4,
  normalize_output_wav=True,
  device="cpu",
)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
#@title Voice Separation model { run: "auto" }

fs = 8000 #@param {type:"integer"}
tag = "lichenda/wsj0_2mix_skim_noncausal" #@param ["espnet/chenda-li-wsj0_2mix_enh_train_enh_conv_tasnet_raw_valid.si_snr.ave", "espnet/chenda-li-wsj0_2mix_enh_train_enh_rnn_tf_raw_valid.si_snr.ave", "lichenda/Chenda_Li_wsj0_2mix_enh_dprnn_tasnet", "lichenda/wsj0_2mix_skim_noncausal", "espnet/Wangyou_Zhang_wsj0_2mix_enh_dc_crn_mapping_snr_raw"]

In [ ]:

import sys
import soundfile
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.enh_inference import SeparateSpeech

d = ModelDownloader()

cfg = d.download_and_unpack(tag)
separate_speech = SeparateSpeech(
  train_config=cfg["train_config"],
  model_file=cfg["model_file"],
  # for segment-wise process on long speech
  segment_size=2.4,
  hop_size=0.8,
  normalize_segment_scale=False,
  show_progressbar=True,
  ref_channel=None,
  normalize_output_wav=True,
  device="cpu",
)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
!gdown --id 1ZCUkd_Lb7pO2rpPr4FqYdtJBZ7JMiInx -O /content/447c020t_1.2106_422a0112_-1.2106.wav

import os
import soundfile
from IPython.display import display, Audio

mixwav, sr = soundfile.read("/content/WhatsApp-Audio-2023-05-08-at-9.22.21-AM (1).wav")
waves_wsj = separate_speech(mixwav[None, ...], fs=sr)

print("Input mixture", flush=True)
display(Audio(mixwav, rate=sr))
print(f"========= Separated speech with model {tag} =========", flush=True)
print("Separated spk1", flush=True)
display(Audio(waves_wsj[0].squeeze(), rate=sr))
print("Separated spk2", flush=True)
display(Audio(waves_wsj[1].squeeze(), rate=sr))

In [ ]:
!pip install pesq pystoi mir_eval


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=255318 sha256=c172fd718d67b2b98abf35efd97f7ded7255f0854e7e8497a19c9fd58333ec66
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7792 sha256=4cfdb4a1174363e54940a63a16b11a86322cd432b3627eb61a0c520561235875
  Stored in directory: /root/.cache/pip/wheels/3b/ca/9e/5b5d6e5e109322303b50d21918ad2bd7d50a2a0775c11e08e8
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100718 sha256=da7de16863f9cae766438e9dd56bc8118f9cf5318078ab55588c3efa9688c241
  Stored in directo

In [ ]:
import mir_eval
import librosa

# Load the original speech signals
spk1, _ = librosa.load('/content/447c020t_1.2106_422a0112_-1.2106.wav', sr=sr)
spk2, _ = librosa.load('/content/447c020t_1.2106_422a0112_-1.2106.wav', sr=sr)

# Load the separated speech signals
separated_spk1, _ = librosa.load('/content/SPK2.wav', sr=sr)
separated_spk2, _ = librosa.load('/content/SPK1.wav', sr=sr)

# Calculate the SDR for each separated speech signal
sdr_spk1, _, _, _ = mir_eval.separation.bss_eval_sources(spk1, separated_spk1)
sdr_spk2, _, _, _ = mir_eval.separation.bss_eval_sources(spk2, separated_spk2)

# Print the SDR for each speaker
print(f"SDR for speaker 1: {float(sdr_spk1):.2f} dB")
print(f"SDR for speaker 2: {float(sdr_spk2):.2f} dB")



SDR for speaker 1: 7.66 dB
SDR for speaker 2: -3.94 dB


In [ ]:
from google.colab import files
from IPython.display import display, Audio
import soundfile
import librosa

# Set the expected sampling rate and duration
sr = 16000
duration = 9  # in seconds
T = int(sr * duration)

# Upload the audio file
uploaded = files.upload()

for file_name in uploaded.keys():
  mixwav_yours, rate = soundfile.read(file_name)
  print(f"Sampling rate of {file_name}: {rate}")
  if rate != sr:
    mixwav_yours = librosa.resample(mixwav_yours, rate, sr)  # Resample the audio file
    rate = sr
  if mixwav_yours.shape[0] != T:
    mixwav_yours = librosa.util.fix_length(mixwav_yours, T)  # Truncate or pad the audio file
  waves_yours = separate_speech(mixwav_yours[None, ...], fs=sr)
  print("Input mixture", flush=True)
  display(Audio(mixwav_yours, rate=sr))
  print(f"========= Separated speech with model {tag} =========", flush=True)
  print("Separated spk1", flush=True)
  display(Audio(waves_yours[0].squeeze(), rate=sr))
  print("Separated spk2", flush=True)
  display(Audio(waves_yours[1].squeeze(), rate=sr))


In [ ]:
import espnet_model_zoo
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

wsj_8k_model_tag="lichenda/wsj_asr_train_asr_transformer_raw_char_8k"

d = ModelDownloader()
speech2text = Speech2Text(
  **d.download_and_unpack(wsj_8k_model_tag),
  device="cpu",
)



Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
text_est = [None, None]
text_est[0], *_ = speech2text(waves_wsj[0].squeeze())[0]
text_est[1], *_ = speech2text(waves_wsj[1].squeeze())[0]
text_m, *_ = speech2text(mixwav)[0]
print("Mix Speech to Text: ", text_m)
print("Separated Speech 1 to Text: ", text_est[0])
print("Separated Speech 2 to Text: ", text_est[1])


/usr/local/lib/python3.10/dist-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


Mix Speech to Text:  THE UNITED SAINT CREATED IT TO DEFEND WHAT IT IN A HIGH REAGANITE IT HAD RATION OFFICIALS ARE RAISING THE ALARM THAT THE FED'S POLICIES TOO TIGHT AND COULD CAUSE A RECESSION NEXT YEAR
Separated Speech 1 to Text:  THE UNITED STATES UNDERTOOK TO DEFEND WESTERN EUROPE AGAINST SOVIET ATTACK SUCH
Separated Speech 2 to Text:  SOME CRITICS INCLUDING HIGH REAGAN ADMINISTRATION OFFICIALS ARE RAISING THE ALARM THAT THE FED'S POLICY IS TOO TIGHT AND COULD CAUSE A RECESSION NEXT YEAR
